In [195]:
#Ignore warnings

import warnings
warnings.filterwarnings('ignore')

#Data handling

import pandas as pd
import numpy as np
import calendar
import datetime

#Plotting

import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

#Time series analysis and modelling

from scipy.ndimage import gaussian_filter
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.tsa.stattools import adfuller,kpss

# Data Cleaning

In [196]:
# Load the data

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df_store = pd.read_csv('data/store.csv')

In [197]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [198]:
# Function to rename columns in lower case
def lower_case(dataframe):
    cols = dataframe.columns.tolist()
    cols = [col.lower() for col in cols]
    dataframe.columns = cols
    return dataframe

In [199]:
lower_case(df_train);
lower_case(df_store);
lower_case(df_test);

In [200]:
# function to change date into datetime
def datetime(dataframe):
    dataframe = dataframe.assign(
        timestamp = lambda x: pd.to_datetime(x['date']),
        year = lambda x: x['timestamp'].dt.year,
        month = lambda x: x['timestamp'].dt.month,
        day = lambda x: x['timestamp'].dt.day,
        dayofyear = lambda x: x['timestamp'].dt.dayofyear)
    return dataframe

In [201]:
# Changing date into datetime object, inserting year, month, day and dayofyear columns
df_train = df_train.assign(
            timestamp = lambda x: pd.to_datetime(x['date']),
            year = lambda x: x['timestamp'].dt.year,
            month = lambda x: x['timestamp'].dt.month,
            day = lambda x: x['timestamp'].dt.day,
            dayofyear = lambda x: x['timestamp'].dt.dayofyear)
df_train.drop("date", inplace=True, axis=1)

In [202]:
# Changing date into datetime object, inserting year, month, day and dayofyear columns
df_test = df_test.assign(
            timestamp = lambda x: pd.to_datetime(x['date']),
            year = lambda x: x['timestamp'].dt.year,
            month = lambda x: x['timestamp'].dt.month,
            day = lambda x: x['timestamp'].dt.day,
            dayofyear = lambda x: x['timestamp'].dt.dayofyear)
df_test.drop("date", inplace=True, axis=1)

In [203]:
df_wdate = df_train.copy()

### Handling NaN's

In [204]:
df_store.isnull().sum()

store                          0
storetype                      0
assortment                     0
competitiondistance            3
competitionopensincemonth    354
competitionopensinceyear     354
promo2                         0
promo2sinceweek              544
promo2sinceyear              544
promointerval                544
dtype: int64

In [205]:
df_store[pd.isnull(df_store.competitiondistance)]

,store,storetype,assortment,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear,promointerval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


In [206]:
# fill NaN with a median value
df_store['competitiondistance'].fillna(df_store['competitiondistance'].median(), inplace = True)
df_store['competitiondistance'].isnull().sum()

0

In [207]:
# fill NaN with a mean value
#df_store['competitiondistance'].fillna(df_store['competitiondistance'].mean(), inplace = True)
#f_store['competitiondistance'].isnull().sum()

In [208]:
# fill NaN with a modus value
#df_store['competitiondistance'].fillna(df_store['competitiondistance'].modus(), inplace = True)
#df_store['competitiondistance'].isnull().sum()

In [209]:
tmp = df_store[pd.isnull(df_store.competitionopensinceyear)]
tmp[tmp.competitiondistance != 0].shape

(354, 10)

Here these stores have a competition in their vicinity ('competitiondistance' =/= 0), but there is no information about the year this competition has been open. This value needs to be imputed in a meaningful way. Or just filled with '0'.

In [210]:
tmp = df_store[pd.isnull(df_store.promo2sinceweek)]
tmp[tmp.promo2 != 0].shape

(0, 10)

There are no stores with information about 'promo2sinceweek' which have 'NaN' in promo2.

In [211]:
# replace NA's by 0

df_store.fillna(0, inplace = True)

In [212]:
print(df_store.isnull().sum())
print('------------------------')
print(df_train.isnull().sum())

store                        0
storetype                    0
assortment                   0
competitiondistance          0
competitionopensincemonth    0
competitionopensinceyear     0
promo2                       0
promo2sinceweek              0
promo2sinceyear              0
promointerval                0
dtype: int64
------------------------
store            0
dayofweek        0
sales            0
customers        0
open             0
promo            0
stateholiday     0
schoolholiday    0
timestamp        0
year             0
month            0
day              0
dayofyear        0
dtype: int64


#### Closed Stores and zero Store Sales

In [213]:
# closed stores

df_train[(df_train['open'] == 0) & (df_train['sales'] == 0)].shape

(172817, 13)

In [214]:
df_train[(df_train['open'] != 0) & (df_train['sales'] == 0)].shape

(54, 13)

- There are 172817 stores, which were closed and had no sales. In addition to 54 open stores, which had no sales at that day.
- We cannot make any predictions for stores, which were closed. Also stores which were open, but had no sales, might have had external influences, such as remodeling.
- To avoid any bias, we should drop these datapoints with 0 sales.

In [215]:
df_train = df_train[(df_train["open"] != 0) & (df_train['sales'] != 0)]
df_train.shape

(844338, 13)

In [216]:
# check for closed stores

df_test[df_test["open"].isnull()]

,id,store,dayofweek,open,promo,stateholiday,schoolholiday,timestamp,year,month,day,dayofyear
479,480,622,4,NaN,1,0,0,2015-09-17,2015,9,17,260
1335,1336,622,3,NaN,1,0,0,2015-09-16,2015,9,16,259
2191,2192,622,2,NaN,1,0,0,2015-09-15,2015,9,15,258
3047,3048,622,1,NaN,1,0,0,2015-09-14,2015,9,14,257
4759,4760,622,6,NaN,0,0,0,2015-09-12,2015,9,12,255
5615,5616,622,5,NaN,0,0,0,2015-09-11,2015,9,11,254
6471,6472,622,4,NaN,0,0,0,2015-09-10,2015,9,10,253
7327,7328,622,3,NaN,0,0,0,2015-09-09,2015,9,9,252
8183,8184,622,2,NaN,0,0,0,2015-09-08,2015,9,8,251
9039,9040,622,1,NaN,0,0,0,2015-09-07,2015,9,7,250


All of these store have no information for 'open', although these days are not a holiday ('stateholiday =/= 1) and are not affected by the closure of schools. They should be open. We need to change the dtyp into integer.

In [217]:
df_test['open'].fillna(1, inplace=True)
df_test['open']= df_test['open'].astype(int)

#### Stateholiday require their input to be uniformly strings or numbers. In our case strings.

In [218]:
df_train['stateholiday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [219]:
df_train.stateholiday.value_counts()

0    731308
0    112120
a       694
b       145
c        71
Name: stateholiday, dtype: int64

In [220]:
df_train['stateholiday'].replace({0:'0'}, inplace=True)
df_train.stateholiday.value_counts()

0    843428
a       694
b       145
c        71
Name: stateholiday, dtype: int64

### Concat Train and Test

In [221]:
df_train['is_train'] = 1
df_test['is_train'] = 0

In [222]:
df_model = pd.concat([df_train, df_test])

### Handling Categorical Data

In [223]:
# Encoding categorical features

df_model['stateholiday'] = LabelEncoder().fit_transform(df_model['stateholiday']) 
df_store['storetype'] = LabelEncoder().fit_transform(df_store['storetype'])
df_store['assortment'] = LabelEncoder().fit_transform(df_store['assortment'])

### Merge Train and Store Data

In [224]:
# Merge df_store and df_train

df = df_train.merge(df_store, how='left', left_on=df_train.store, right_on=df_store.store)
df.drop(['key_0', 'store_y'], axis=1, inplace=True)
df = df.rename(columns={'store_x':'store'})

In [225]:
# withe "date"

df_wdate = df_wdate.merge(df_store, how='left', left_on=df_wdate.store, right_on=df_store.store)
df_wdate.drop(['key_0', 'store_y'], axis=1, inplace=True)
df_wdate = df_wdate.rename(columns={'store_x':'store'})

### Store the cleaned Dataframes

In [226]:
%store df_model df_store df df_wdate

Stored 'df_model' (DataFrame)
Stored 'df_store' (DataFrame)
Stored 'df' (DataFrame)
Stored 'df_wdate' (DataFrame)
